### 1. 시퀀스-투-시퀀스(Sequence-to-Sequence)

In [1]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
path = os.getcwd()
with zipfile.ZipFile('fra-eng.zip', 'r') as zip_ref:
    zip_ref.extractall(path)

In [3]:
lines = pd.read_csv('fra.txt', names = ['src', 'tar', 'lic'], sep = '\t')
del lines['lic']
print("전체 샘플의 개수 : ", len(lines))

전체 샘플의 개수 :  232736


In [4]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 저장
lines.sample(10)

,src,tar
34975,I've got to go now.,Je dois partir maintenant.
20362,How tall you are!,Que vous êtes grandes !
14164,You're invited.,Vous êtes invitée.
36269,That was mentioned.,Ce fut mentionné.
45004,Tom works all night.,Tom travaille toute la nuit.
58630,I'm not a kid anymore.,Je ne suis plus un enfant.
45451,We're scared of Tom.,Nous avons peur de Tom.
19671,Don't be so lazy.,Ne soyez pas si paresseux !
37707,We had fun with it.,Nous nous en sommes amusés.
33063,He still loves her.,Il l'aime toujours.


In [5]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
2804,You decide.,"\t Toi, décide. \n"
11271,I feel seasick.,\t J'ai le mal de mer. \n
16066,I'll be on time.,\t Je serai à l'heure. \n
32693,Fill out this form.,\t Remplissez ce formulaire. \n
45797,Who broke the chair?,\t Qui a cassé la chaise ? \n
32316,Do I have to study?,\t Dois-je étudier ? \n
19119,You're so brave.,\t Tu es très courageuse. \n
1962,Head south.,\t Dirigez-vous vers le sud. \n
55084,Are you leaving today?,\t Pars-tu aujourd'hui ? \n
25212,You're very sick.,\t Tu es gravement malade. \n


In [6]:
# 문자 집합 구축
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [7]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

source 문장의 char 집합 : 80
target 문장의 char 집합 : 102


In [8]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [9]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, 'ï': 77, '’': 78, '€': 79}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, ',': 10, '-': 11, '.': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 3

In [10]:
encoder_input = []

# 1개의 문장
for line in lines.src:
  encoded_line = []
  # 각 줄에서 1개의 char
  for char in line:
    # 각 char을 정수로 변환
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩 :',encoder_input[:5])

source 문장의 정수 인코딩 : [[30, 64, 10], [30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10]]


In [11]:
decoder_input = []
for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])

target 문장의 정수 인코딩 : [[1, 3, 46, 50, 3, 4, 3, 2], [1, 3, 37, 50, 67, 52, 57, 54, 12, 3, 2], [1, 3, 29, 63, 3, 67, 64, 70, 69, 54, 3, 4, 3, 2], [1, 3, 26, 64, 70, 56, 54, 3, 4, 3, 2], [1, 3, 43, 50, 61, 70, 69, 3, 4, 3, 2]]


In [12]:
decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 46, 50, 3, 4, 3, 2], [3, 37, 50, 67, 52, 57, 54, 12, 3, 2], [3, 29, 63, 3, 67, 64, 70, 69, 54, 3, 4, 3, 2], [3, 26, 64, 70, 56, 54, 3, 4, 3, 2], [3, 43, 50, 61, 70, 69, 3, 4, 3, 2]]


In [13]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)


source 문장의 최대 길이 : 22
target 문장의 최대 길이 : 76


In [14]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [15]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

#### 2) 교사 강요(Teacher forcing)
#### 3) seq2seq 기계 번역기 훈련시키기

In [16]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [17]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs은 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태.
encoder_states = [state_h, state_c]

In [18]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [19]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 231s 301ms/step - loss: 1.1277 - val_loss: 0.7671
Epoch 2/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 226s 301ms/step - loss: 0.5921 - val_loss: 0.6557
Epoch 3/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 221s 295ms/step - loss: 0.5139 - val_loss: 0.5970
Epoch 4/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 217s 289ms/step - loss: 0.4631 - val_loss: 0.5500
Epoch 5/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 208s 276ms/step - loss: 0.4254 - val_loss: 0.5148
Epoch 6/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 207s 276ms/step - loss: 0.3978 - val_loss: 0.4897
Epoch 7/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 204s 272ms/step - loss: 0.3769 - val_loss: 0.4683
Epoch 8/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 204s 272ms/step - loss: 0.3581 - val_loss: 0.4566
Epoch 9/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 215s 287ms/step - loss: 0.3449 - val_loss: 0.4390
Epoch 10/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 219s 291ms/step - loss: 0.3326 - val_loss: 0.4264
Epoch 11/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 226s 301ms/step - loss: 0.3234 - val_loss: 0.4164
Epoch 12

#### 4) seq2seq 기계 번역기 동작시키기

In [20]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)